In [11]:
#!pip install sodapy
#!pip install pandas

In [12]:
import pandas as pd
from sodapy import Socrata

In [13]:
client = Socrata("opendata.utah.gov", None)
results = client.get("herb-zqda", limit=252500)
df = pd.DataFrame.from_records(results)
df


,crash_id,crash_datetime,route,milepoint,lat_utm_y,long_utm_x,main_road_name,city,county_name,crash_severity_id,...,domestic_animal_related,overturn_rollover,commercial_motor_veh_involved,teenage_driver_involved,older_driver_involved,night_dark_condition,single_vehicle,distracted_driving,drowsy_driving,roadway_departure
0,11281387,2019-02-08T10:56:00.000,2258,4.56,4513124.085,422620.0512,900 WEST,SALT LAKE CITY,SALT LAKE,2,...,False,False,True,False,False,False,False,False,False,True
1,11282089,2019-12-24T12:48:00.000,2082,2.702,4483700.743,427458.6694,1215 E HIGHLAND DR,DRAPER,SALT LAKE,1,...,False,False,False,True,False,False,False,False,False,False
2,11282092,2019-12-28T20:42:00.000,2062,0.351,4486609.366,426779.0152,1022 E DRAPER PKY,DRAPER,SALT LAKE,1,...,False,False,False,True,False,True,False,False,False,False
3,11282098,2019-12-07T00:46:00.000,2048,0.438,4484350.913,426397.6578,13463 S FORT ST,DRAPER,SALT LAKE,3,...,False,True,False,False,False,True,True,False,False,False
4,11282101,2019-12-18T17:29:00.000,2068,0.796,4485217.604,427714.0392,13043 S 1300 E,DRAPER,SALT LAKE,1,...,False,False,False,True,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252495,11304083,2019-11-21T23:17:00.000,353484,0.1,4484305.829,412209.1538,6094 W CHAMPLAIN WY,HERRIMAN,SALT LAKE,1,...,False,False,False,False,False,True,False,False,False,True
252496,11304174,2019-10-22T19:09:00.000,2068,1.464,4486291.662,427726.2553,1300 E CONSTITUTION AVE,DRAPER,SALT LAKE,2,...,False,False,False,False,False,False,True,False,False,False
252497,11304628,2019-10-25T17:30:00.000,2033,0.998,4483292.486,425009.0526,14000 S BANGERTER PKY,DRAPER,SALT LAKE,3,...,False,False,False,False,False,False,True,False,False,False
252498,11313074,2019-11-06T18:54:00.000,352012,0.1,4486037.269,428211.6975,12629 S RELATION ST,DRAPER,SALT LAKE,1,...,False,False,False,False,True,True,False,False,False,False


In [14]:
df.dropna()
df.drop(columns=["route","crash_id","lat_utm_y","long_utm_x","main_road_name","county_name","city"],inplace=True)

In [15]:
for col in df.columns:
    print(df[col].isnull().sum())

2
23
4
1939
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4


In [16]:
#SINCE NO COLUMNS HAD MORE THAN A SIGNIFICANT NUMBER OF NULL VALUES THERE IS NO REASON TO DROP ANY HERE
#DROPNA() DIDN'T END UP DOING ANYTHING SO I AM LOOKING TO REMOVE NaN
# for col in df.columns:
#     print(df[col].isnull().sum())
#A column had a total of 1939 missing - rerun to delete all

#ITERATE THROUGH EVERY ROW (SEE IF NULL) TO SEE WHAT ROWS CAN BE DELETE
for i , r in df.iterrows():
   if pd.isnull(r).any():
      df.drop(i,inplace = True)

In [17]:
#VERIFY THAT NO COLUMNS HAVE NULL VALUES LEFT (SUM OF NULL VALUES)
for col in df.columns:
    print(df[col].isnull().sum())

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [18]:
df.dtypes

crash_datetime                   object
milepoint                        object
crash_severity_id                object
work_zone_related                object
pedestrian_involved              object
bicyclist_involved               object
motorcycle_involved              object
improper_restraint               object
unrestrained                     object
dui                              object
intersection_related             object
wild_animal_related              object
domestic_animal_related          object
overturn_rollover                object
commercial_motor_veh_involved    object
teenage_driver_involved          object
older_driver_involved            object
night_dark_condition             object
single_vehicle                   object
distracted_driving               object
drowsy_driving                   object
roadway_departure                object
dtype: object

In [19]:
#THE API SENDS EVERY DATAPOINT AS A STRING. CAST DATA FOR BETTER CLEANING

#DATETIME - ONLY CRASH_DATETIME
df['crash_datetime'] = pd.to_datetime(df['crash_datetime'], format='%Y-%m-%dT%H:%M:%S.%f')
#INTEGERS
df['crash_severity_id'] = df['crash_severity_id'].astype(int)
df['milepoint'] = df['milepoint'].astype(float)
#EVERYTHING ELSE
for col in df.columns: 
    if col == 'crash_severity_id' or col =='milepoint' or col =='crash_datetime':
        temp = 0
    else:
        df[col] = df[col].astype(int)
# IN THIS CASE OBJECTS are going to be treated still like bool values
df.dtypes

crash_datetime                   datetime64[ns]
milepoint                               float64
crash_severity_id                         int64
work_zone_related                         int64
pedestrian_involved                       int64
bicyclist_involved                        int64
motorcycle_involved                       int64
improper_restraint                        int64
unrestrained                              int64
dui                                       int64
intersection_related                      int64
wild_animal_related                       int64
domestic_animal_related                   int64
overturn_rollover                         int64
commercial_motor_veh_involved             int64
teenage_driver_involved                   int64
older_driver_involved                     int64
night_dark_condition                      int64
single_vehicle                            int64
distracted_driving                        int64
drowsy_driving                          

In [20]:
#SINCE MILEPOINT HAS MANY POINTS BETWEEN 1-6 LETS BIN
len = df.shape[0]
count = 0
#setting a milepoint position variable
p = 1
for x in range(0,len):
    if df.iat[count,p] < .5:
        df.iat[count,p] = 0
    elif df.iat[count,p] < 1:
        df.iat[count,p] = 1
    elif df.iat[count,p] < 1.5:
        df.iat[count,p] = 1.5
    elif df.iat[count,p] < 2:
        df.iat[count,p] = 2
    elif df.iat[count,p] < 2.5:
        df.iat[count,p] = 2.5
    elif df.iat[count,p] < 3.5:
        df.iat[count,p] = 3.5
    elif df.iat[count,p] < 4:
        df.iat[count,p] = 4
    elif df.iat[count,p] < 4.5:
        df.iat[count,p] = 4.5
    elif df.iat[count,p] < 5:
        df.iat[count,p] = 5
    elif df.iat[count,p] < 5.5:
        df.iat[count,p] = 5.5
    else:
        df.iat[count,p] = 6 
    count += 1      
df

,crash_datetime,milepoint,crash_severity_id,work_zone_related,pedestrian_involved,bicyclist_involved,motorcycle_involved,improper_restraint,unrestrained,dui,...,domestic_animal_related,overturn_rollover,commercial_motor_veh_involved,teenage_driver_involved,older_driver_involved,night_dark_condition,single_vehicle,distracted_driving,drowsy_driving,roadway_departure
0,2019-02-08 10:56:00,5.0,2,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,1
1,2019-12-24 12:48:00,3.5,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,2019-12-28 20:42:00,0.0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
3,2019-12-07 00:46:00,0.0,3,0,0,0,0,0,0,1,...,0,1,0,0,0,1,1,0,0,0
4,2019-12-18 17:29:00,1.0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252495,2019-11-21 23:17:00,0.0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,1
252496,2019-10-22 19:09:00,1.5,2,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
252497,2019-10-25 17:30:00,1.0,3,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
252498,2019-11-06 18:54:00,0.0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0


In [21]:
#Need to import datetime from datetime explicitely
from datetime import datetime
#Add the columns we can create
pHour = 1
pDay = 2
pMonth = 3
df.insert(pHour,'hour',0)
df.insert(pDay,"weekday",0)
df.insert(pMonth,'month',0)

#Break out the specific data that could be used (DATE, HOUR, WEEKDAY)
count = 0
for x in range(0,len):
    date = df.iat[count,0]   
    df.iat[count,pHour] = date.hour
    df.iat[count,pDay] = date.weekday()
    df.iat[count,pMonth] = date.month
    count += 1
df.drop(columns='crash_datetime',inplace=True)
df

,hour,weekday,month,milepoint,crash_severity_id,work_zone_related,pedestrian_involved,bicyclist_involved,motorcycle_involved,improper_restraint,...,domestic_animal_related,overturn_rollover,commercial_motor_veh_involved,teenage_driver_involved,older_driver_involved,night_dark_condition,single_vehicle,distracted_driving,drowsy_driving,roadway_departure
0,10,4,2,5.0,2,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
1,12,1,12,3.5,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,20,5,12,0.0,1,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
3,0,5,12,0.0,3,0,0,0,0,0,...,0,1,0,0,0,1,1,0,0,0
4,17,2,12,1.0,1,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252495,23,3,11,0.0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
252496,19,1,10,1.5,2,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
252497,17,4,10,1.0,3,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
252498,18,2,11,0.0,1,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0


In [27]:
df['hour'].max()

23

In [23]:
#df.to_csv('APICleanedCrashes.csv')